# Feature Selection

In this practical, you will become familiarized with some basic feature selection methods implemented in scikit-learn. Consider the prostate dataset that is attached to this practical. You are asked to:

1. Estimate the performance of the nearest neighbor classifier on this dataset using 10-fold cross validation when all the features are used for prediction. The number of neighbors should be chosen using an inner cross-validation procedure. You can use 5-fold cross validation for this.

2. Estimate the performance of the nearest neighbor classifier on the same dataset when using a feature selection technique based on the F-score (ANOVA) that picks up the 10 most relevant features. Use the same cross-validation methods as in the previous step.

3. Repeat the previous experiment but when a random forest is used to pick up the 10 most relevant features. Use an initial filtering method based on the F-score to keep only the 20% most promising features.

4. What feature selection method performs best? Can you explain why?

Now we will address the problem of analyzing the trade-off between interpretability and prediction accuracy. For this, you are asked to:

1. Estimate the performance of the nearest neighbor classifier with K=3 as a function of the features used for prediction. Use a 10-times 10-fold cross-validation method and plot the results obtained. That is prediction error vs. the number of features used for prediction. Use the F-score for feature selection. Report results from 1 feature to 200 features. Not all features need to be explored. Use a higher resolution when you are closer to 1 feature.

2. Repeat that process when the feature selection is done externally to the cross-validation loop using all the available data. Include these results in the previous plot.
3. Are the two estimates obtained similar? What are their differences? If they are different try to explain why this is the case.

4. By taking a look at these results, what is the optimal number of features to use in this dataset?

5. Given the results obtained in this part of the practical, you are asked to indicate which particular features should be used for prediction on this dataset. Include a list with them. Take a look at the documentation of SelectKBest from scikit-learn to understand how to do this. Use all available data to provide such a list of features. 

In [59]:
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib as mpl
from matplotlib import colors
import seaborn as sns; sns.set()
import scipy.stats as stats
import scipy as sp
from scipy import linalg
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, KFold, GridSearchCV, cross_val_score
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest, f_classif, SelectFromModel
from sklearn.metrics import accuracy_score, make_scorer, confusion_matrix, classification_report, precision_score

## Question 1

Estimate the performance of the nearest neighbor classifier on this dataset using 10-fold cross validation when all the features are used for prediction. The number of neighbors should be chosen using an inner cross-validation procedure. You can use 5-fold cross validation for this.

### Response 1

We obtain a Mean Error of $0.211$ with std mean error of $0.012$

In [74]:
N_neighbors = [1, 3, 5, 7, 11, 15, 20, 25, 30]

def get_scaler(X_train):
    return preprocessing.StandardScaler().fit(X_train)
    
def make_grid(n_splits):
    pipeline = Pipeline([ ('knn', KNeighborsClassifier()) ])
    param_grid = { 'knn__n_neighbors': N_neighbors }
    skfold = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=1, random_state=0)
    gridcv = GridSearchCV(pipeline, cv=skfold, n_jobs=1, param_grid=param_grid, scoring=make_scorer(accuracy_score))
    return gridcv

def estimate_n_neighbors(X_train, y_train, X_test, y_test, gridcv, do_plot=False, verbose=True):
    result = gridcv.fit(X_train, y_train)
    accuracies = gridcv.cv_results_['mean_test_score']
    std_accuracies = gridcv.cv_results_['std_test_score']
    N_neighbors_selected = N_neighbors[np.argmax(accuracies)]
    if do_plot:
        test_accuracies = np.ones(len(N_neighbors))
        for i in range(len(N_neighbors)):
            knn = KNeighborsClassifier(n_neighbors = N_neighbors[ i ])
            knn.fit(X_train, y_train)
            test_accuracies[ i ] = accuracy_score(knn.predict(X_test), y_test)
        plt.figure()
        line1, = plt.plot(N_neighbors, accuracies, 'o-', color="g")
        line2, = plt.plot(N_neighbors, test_accuracies, 'x-', color="r")
        plt.fill_between(N_neighbors, accuracies - std_accuracies / np.sqrt(10), \
            accuracies + std_accuracies / np.sqrt(10), alpha=0.1, color="g")
        plt.grid()
        plt.title("Different number of neighbors for KNN")
        plt.xlabel('Number of neighbors')
        plt.xticks(np.array(N_neighbors))
        plt.ylabel('Prostate Cancer - Classification Accuracy')
        plt.ylim((0.5, 1.0))
        legend_handles = [ mlines.Line2D([], [], color='g', marker='o', \
                                  markersize=15, label='CV-estimate'), \
                        mlines.Line2D([], [], color='r', marker='x', \
                                  markersize=15, label='Test set estimate')]
        plt.legend(handles=legend_handles, loc = 3)
        plt.show()
    
    if verbose: print("Selected optimal number of neighbors", N_neighbors_selected)
    return N_neighbors_selected    

In [71]:
# Load data
data = pd.read_csv('prostate.csv', header = 0)
X = data.values[ :, 0:-1].astype(np.float)
y = (data.values[ :, -1] == 1).astype(np.int)

# Construct pipeline for n_neighbors inner estimation
gridcv = make_grid(n_splits=5)

In [76]:
# Evaluating performance for all features
(_, n_features) = X.shape
print("Number of features", n_features)
n_splits = 10
n_repeats = 10
random_state = 1
rkf = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=random_state)
split = 0
knn_errors = np.zeros(n_splits * n_repeats)
for train_index, test_index in rkf.split(X, y):
    print("Partition", split)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Standardization
    scaler = get_scaler(X_train)
    scaler.fit(X_train, y_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Compute hyperparam
    n_neighbors = estimate_n_neighbors(X_train_scaled, y_train, X_test_scaled, y_test,\
                                       gridcv, do_plot=False, verbose=True)
    # Training & Scoring
    knn = KNeighborsClassifier(n_neighbors=n_neighbors)
    knn.fit(X_train_scaled, y_train)
    knn_errors[split] = 1.0 - np.mean(knn.predict(X_test_scaled) == y_test)
    split += 1
print()
print("Errors", knn_errors)
print()
print("Mean", knn_errors.mean(), "std mean error", (np.std(knn_errors) / np.sqrt(len(knn_errors))))
print()

Number of features 12625
Partition 0
Selected optimal number of neighbors 7
Partition 1
Selected optimal number of neighbors 7
Partition 2
Selected optimal number of neighbors 7
Partition 3
Selected optimal number of neighbors 11
Partition 4
Selected optimal number of neighbors 7
Partition 5
Selected optimal number of neighbors 3
Partition 6
Selected optimal number of neighbors 3
Partition 7
Selected optimal number of neighbors 5
Partition 8
Selected optimal number of neighbors 3
Partition 9
Selected optimal number of neighbors 7
Partition 10
Selected optimal number of neighbors 7
Partition 11
Selected optimal number of neighbors 3
Partition 12
Selected optimal number of neighbors 11
Partition 13
Selected optimal number of neighbors 3
Partition 14
Selected optimal number of neighbors 11
Partition 15
Selected optimal number of neighbors 7
Partition 16
Selected optimal number of neighbors 11
Partition 17
Selected optimal number of neighbors 5
Partition 18
Selected optimal number of neigh

## Question 2

Estimate the performance of the nearest neighbor classifier on the same dataset when using a feature selection technique based on the F-score (ANOVA) that picks up the 10 most relevant features. Use the same cross-validation methods as in the previous step.

### Response 2

We obtain a Mean Error of $0.088$ with std mean error of $0.008$

In [86]:
n_features = 10
print("Number of features", n_features)
n_splits = 10
n_repeats = 10
random_state = 2
rkf = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=random_state)
split = 0
knn_errors = np.zeros(n_splits * n_repeats)
for train_index, test_index in rkf.split(X, y):
    print("Partition", split)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Standardization
    scaler = get_scaler(X_train)
    scaler.fit(X_train, y_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Filtering
    filtering = SelectKBest(f_classif, k = n_features)
    filtering.fit(X_train_scaled, y_train)
    X_train_filtered = filtering.transform(X_train_scaled)
    X_test_filtered = filtering.transform(X_test_scaled)
    
    # Compute hyperparam
    n_neighbors = estimate_n_neighbors(X_train_filtered, y_train, X_test_filtered, y_test,\
                                       gridcv, do_plot=False, verbose=True)
    # Training & Scoring
    knn = KNeighborsClassifier(n_neighbors=n_neighbors)
    knn.fit(X_train_filtered, y_train)
    knn_errors[split] = 1.0 - np.mean(knn.predict(X_test_filtered) == y_test)
    split += 1
print("Errors", knn_errors)
print()
print("Mean", knn_errors.mean(), "std mean error", (np.std(knn_errors) / np.sqrt(len(knn_errors))))
print()

Number of features 10
Partition 0
Selected optimal number of neighbors 3
Partition 1
Selected optimal number of neighbors 3
Partition 2
Selected optimal number of neighbors 7
Partition 3
Selected optimal number of neighbors 25
Partition 4
Selected optimal number of neighbors 3
Partition 5
Selected optimal number of neighbors 1
Partition 6
Selected optimal number of neighbors 3
Partition 7
Selected optimal number of neighbors 5
Partition 8
Selected optimal number of neighbors 5
Partition 9
Selected optimal number of neighbors 5
Partition 10
Selected optimal number of neighbors 5
Partition 11
Selected optimal number of neighbors 1
Partition 12
Selected optimal number of neighbors 7
Partition 13
Selected optimal number of neighbors 3
Partition 14
Selected optimal number of neighbors 11
Partition 15
Selected optimal number of neighbors 5
Partition 16
Selected optimal number of neighbors 7
Partition 17
Selected optimal number of neighbors 11
Partition 18
Selected optimal number of neighbors

## Question 3

Repeat the previous experiment but when a random forest is used to pick up the 10 most relevant features. Use an initial filtering method based on the F-score to keep only the 20% most promising features.


### Response 3

We obtain a Mean Error of $0.080$ with std mean error of $0.008$

In [85]:
n_features = 10
print("Number of features", n_features)
n_splits = 10
n_repeats = 10
random_state = 2
rkf = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=random_state)
split = 0
knn_errors = np.zeros(n_splits * n_repeats)
rf_selection =  SelectFromModel(RandomForestClassifier(n_estimators = 2000,\
                                                       random_state = random_state), threshold = 0.0)
for train_index, test_index in rkf.split(X, y):
    print("Partition", split)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Standardization
    scaler = get_scaler(X_train)
    scaler.fit(X_train, y_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Filtering pass 1 via F-score, only 20%
    inner_features = X_train.shape[1]
    features_to_retain = int(20*inner_features/100)
    filtering = SelectKBest(f_classif, k = features_to_retain)
    filtering.fit(X_train_scaled, y_train)
    X_train_filtered_F = filtering.transform(X_train_scaled)
    X_test_filtered_F = filtering.transform(X_test_scaled)
    n_features_reduction_F = X_train_filtered_F.shape[1]
    print("Features reduced to (first pass)", n_features_reduction_F)
    
    # Filtering random forest
    rf_selection.fit(X_train_filtered_F, y_train)
    rf_selection.threshold = -1.0 * np.sort(-1.0 * rf_selection.estimator_.feature_importances_)[n_features-1]
    X_train_filtered_F_R = rf_selection.transform(X_train_filtered_F)
    X_test_filtered_F_R = rf_selection.transform(X_test_filtered_F)
    n_features_reduction = X_train_filtered_F_R.shape[1]
    print("Features reduced to (second pass)", n_features_reduction)
    
     # Compute hyperparam
    n_neighbors = estimate_n_neighbors(X_train_filtered_F_R, y_train, X_test_filtered_F_R, y_test,\
                                       gridcv, do_plot=False, verbose=True)
    
    # Training & Scoring
    knn = KNeighborsClassifier(n_neighbors=n_neighbors)
    knn.fit(X_train_filtered_F_R, y_train)
    knn_errors[split] = 1.0 - np.mean(knn.predict(X_test_filtered_F_R) == y_test)
    split += 1
print()
print("Errors", knn_errors)
print()
print("Mean", knn_errors.mean(), "std mean error", (np.std(knn_errors) / np.sqrt(len(knn_errors))))
print()
print()

Number of features 10
Partition 0
Features reduced to (first pass) 2525
Features reduced to (second pass) 10
Selected optimal number of neighbors 3
Partition 1
Features reduced to (first pass) 2525
Features reduced to (second pass) 10
Selected optimal number of neighbors 5
Partition 2
Features reduced to (first pass) 2525
Features reduced to (second pass) 10
Selected optimal number of neighbors 3
Partition 3
Features reduced to (first pass) 2525
Features reduced to (second pass) 10
Selected optimal number of neighbors 25
Partition 4
Features reduced to (first pass) 2525
Features reduced to (second pass) 10
Selected optimal number of neighbors 7
Partition 5
Features reduced to (first pass) 2525
Features reduced to (second pass) 10
Selected optimal number of neighbors 5
Partition 6
Features reduced to (first pass) 2525
Features reduced to (second pass) 10
Selected optimal number of neighbors 15
Partition 7
Features reduced to (first pass) 2525
Features reduced to (second pass) 10
Selecte

Features reduced to (second pass) 10
Selected optimal number of neighbors 5
Partition 65
Features reduced to (first pass) 2525
Features reduced to (second pass) 10
Selected optimal number of neighbors 1
Partition 66
Features reduced to (first pass) 2525
Features reduced to (second pass) 10
Selected optimal number of neighbors 3
Partition 67
Features reduced to (first pass) 2525
Features reduced to (second pass) 10
Selected optimal number of neighbors 15
Partition 68
Features reduced to (first pass) 2525
Features reduced to (second pass) 10
Selected optimal number of neighbors 7
Partition 69
Features reduced to (first pass) 2525
Features reduced to (second pass) 10
Selected optimal number of neighbors 3
Partition 70
Features reduced to (first pass) 2525
Features reduced to (second pass) 10
Selected optimal number of neighbors 3
Partition 71
Features reduced to (first pass) 2525
Features reduced to (second pass) 10
Selected optimal number of neighbors 3
Partition 72
Features reduced to (

## Question 4
What feature selection method performs best? Can you explain why?

### Response 4

ANOVA performs marginally worst than Random Forest, with overlapping confidence intervals:

- Anova $0.088 \pm 0.008$.
- Random Forest $0.080 \pm 0.008$.

Decision trees are prone to be more sticky to the data, because of his very hard dependences of initial selection nodes, because it chooses the best result at a given step, but does not ensure that this is the optimal decision of the whole route to the leaf node. Random forests mitigate this problem but the algorithm needs enough features in order to make more effective the randomization. Perhaps the number of features is too low in this case.

In fact, if we rise the number of selected features to $20$, Random Forest get more advantage over Anova, but yet very slightly.

- Anova $0.083 \pm 0.008$.
- Random Forest $0.074 \pm 0.008$.

# Outputs

In [57]:
%%bash
jupyter nbconvert --to=latex --template=~/report.tplx feature_selection_v2.ipynb 1> /dev/null
pdflatex -shell-escape feature_selection_v2 1> /dev/null
jupyter nbconvert --to html_with_toclenvs feature_selection_v2.ipynb 1> /dev/null

[NbConvertApp] Converting notebook feature_selection_v2.ipynb to latex
[NbConvertApp] Support files will be in feature_selection_v2_files/
[NbConvertApp] Making directory feature_selection_v2_files
[NbConvertApp] Writing 41604 bytes to feature_selection_v2.tex
[NbConvertApp] Converting notebook feature_selection_v2.ipynb to html_with_toclenvs
[NbConvertApp] Writing 371529 bytes to feature_selection_v2.html
